In [ ]:
# Os dados importados limpos se referem as notas desconsiderando os casos em que ha zero em alguma das areas de conhecimento,
# com excessao da redacao, que pode ter sido zero

import pandas as pd
df = pd.read_csv("../Data/dados_enem_2022_limpo.csv")

df

,NU_INSCRICAO,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ESCOLA,NO_MUNICIPIO_ESC,SG_UF_ESC,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TP_LINGUA,NU_NOTA_REDACAO,NU_IDADE,NOTA_MEDIA
0,2.100570e+11,F,Solteiro(a),Parda,Brasileiro(a),Publica,Aracati,CE,436.7,420.0,512.6,430.4,Espanhol,560.0,20.0,471.94
1,2.100560e+11,F,Solteiro(a),Branca,Brasileiro(a),Publica,São Paulo,SP,819.5,684.9,657.8,905.5,Ingles,900.0,18.0,793.54
2,2.100560e+11,M,Solteiro(a),Parda,Brasileiro(a),Publica,Goianésia,GO,442.4,579.2,582.7,551.9,Ingles,740.0,17.0,579.24
3,2.100550e+11,F,Solteiro(a),Parda,Brasileiro(a),Publica,Santarém,PA,461.4,485.9,448.9,433.6,Espanhol,520.0,18.0,469.96
4,2.100560e+11,M,Solteiro(a),Branca,Brasileiro(a),Publica,Teresópolis,RJ,427.8,485.0,557.5,438.0,Espanhol,540.0,18.0,489.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191308,2.100550e+11,F,Solteiro(a),Parda,Brasileiro(a),Publica,Maranguape,CE,496.8,580.1,575.9,545.9,Espanhol,840.0,18.0,607.74
191309,2.100580e+11,F,Solteiro(a),Parda,Brasileiro(a),Publica,Fortaleza,CE,388.8,510.0,488.0,502.1,Espanhol,600.0,17.0,497.78
191310,2.100560e+11,F,Solteiro(a),Preta,Brasileiro(a),Publica,Jaguaribe,CE,445.3,516.7,571.6,500.4,Espanhol,700.0,18.0,546.80
191311,2.100570e+11,F,Solteiro(a),Parda,Brasileiro(a),Publica,Morrinhos,CE,408.0,445.8,419.8,406.1,Espanhol,720.0,18.0,479.94
